# Players Stats Analysis
The data here are clean and prepared to be use as a input of a [Tableau dashboard](https://public.tableau.com/profile/axel.rasse#!/vizhome/SoccerprojectV2/Dashboard)
We proceed basic task such as handling missing data, converting data type, looking for duplicated values... After a bit of exploration, some new ideas pop up and we have experiment some attributes combinations such as grouping game type in four buckets or creating new features : goals and assists per minutes of every players.

### Summary
1. [Load](#load)
2. [Clean](#clean)
3. [Data Exploration](#explore)
4. [Save](#save)

___
# 1. Load
<a id='load'></a>

In [82]:
import pandas as pd
import numpy as np
players_stats = pd.read_csv('../data/players_database.csv', parse_dates=True)
top_scorer = pd.read_json('../data/250_best_scorer.json')

* **top_scorer.csv** rank the top 250 scorers 

* **players_stat.csv** detailed data of the best 250 scorers _(date, minutes played, goals, assists..)_

## a. Top scorer

In [83]:
top_scorer = top_scorer.sort_values(by='goals', ascending=False)
top_scorer

,name,club,age,league,nationality,match_played,goals,assists,total
0,Ciro Immobile,Lazio,30,Italy,Italy,26,27,7,34
13,Erling Haaland,Bor. Dortmund,19,Germany,Norway,22,25,8,33
3,Robert Lewandowski,Bayern Munich,31,Germany,Poland,23,25,3,28
6,Shon Weissman,Wolfsberger AC,24,Austria,Israel,21,22,3,25
1,Odsonne Edouard,Celtic,22,Scotland,France,27,21,12,33
...,...,...,...,...,...,...,...,...,...
147,Luis Alberto,Lazio,27,Italy,Spain,25,4,13,17
231,Miroslav Stevanovic,Servette FC,29,Switzerland,Bosnia-Herzegovina,21,4,11,15
243,Dan Nistor,CS U Craiova,31,Romania,Romania,22,4,11,15
119,Christopher Nkunku,RB Leipzig,22,Germany,France,23,4,14,18


In [84]:
top_scorer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250 entries, 0 to 244
Data columns (total 9 columns):
name            250 non-null object
club            250 non-null object
age             250 non-null int64
league          250 non-null object
nationality     250 non-null object
match_played    250 non-null int64
goals           250 non-null int64
assists         250 non-null int64
total           250 non-null int64
dtypes: int64(5), object(4)
memory usage: 19.5+ KB


## b. Players Stats

In [85]:
players_stats[:5]

,date,comp,round,opponent,minutes,goals,assists,shots_total,shots_on_target,crosses,fouled,pens_made,pens_att,xg,npxg,xa,name
0,2019-08-25,Serie A,Matchweek 1,Sampdoria,90.0,2,0.0,5.0,4.0,0.0,3.0,0,0,1.3,1.3,0.1,Ciro Immobile
1,2019-09-01,Serie A,Matchweek 2,Roma,90.0,0,1.0,7.0,1.0,0.0,1.0,0,0,0.6,0.6,0.7,Ciro Immobile
2,2019-09-08,Euro Qualifying,Group stage,Finland,76.0,1,0.0,5.0,2.0,0.0,0.0,0,0,0.5,0.5,0.1,Ciro Immobile
3,2019-09-15,Serie A,Matchweek 3,SPAL,90.0,1,0.0,0.0,0.0,1.0,0.0,1,1,0.8,0.0,0.1,Ciro Immobile
4,2019-09-22,Serie A,Matchweek 4,Parma,63.0,1,0.0,3.0,2.0,2.0,0.0,0,0,0.2,0.2,0.2,Ciro Immobile


___
# 2. Clean
<a id='clean'></a>

In [86]:
players_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5189 entries, 0 to 5188
Data columns (total 17 columns):
date               5189 non-null object
comp               4325 non-null object
round              5189 non-null object
opponent           5189 non-null object
minutes            5179 non-null float64
goals              5189 non-null int64
assists            5100 non-null float64
shots_total        3990 non-null float64
shots_on_target    3990 non-null float64
crosses            3990 non-null float64
fouled             3990 non-null float64
pens_made          5189 non-null int64
pens_att           5189 non-null int64
xg                 1759 non-null float64
npxg               1759 non-null float64
xa                 1759 non-null float64
name               5189 non-null object
dtypes: float64(9), int64(3), object(5)
memory usage: 689.3+ KB


↳ We might have a lot of missing values, especially in the expect fields (xg, npxg, xa). Maybe player from little league do not have this kind of technology 🤔 

Some of the columns are in the wrong type - for instance, assists, need to be in integer

## a. Missing Values

In [87]:
players_stats['xg'].isnull().value_counts()

True     3430
False    1759
Name: xg, dtype: int64

In [88]:
players_stats[players_stats['xg'].isnull()][-3:]

,date,comp,round,opponent,minutes,goals,assists,shots_total,shots_on_target,crosses,fouled,pens_made,pens_att,xg,npxg,xa,name
5185,2020-02-12,First Division A,Regular Season,Standard Liège,90.0,0,0.0,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,Ruud Vormer
5186,2020-02-15,First Division A,Regular Season,Waasland-Bev,90.0,0,0.0,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,Ruud Vormer
5188,2020-02-23,First Division A,Regular Season,Charleroi,90.0,0,0.0,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,Ruud Vormer


↳ Leave the NaN values like that for the moments

In [89]:
players_stats['assists'].isnull().value_counts()

False    5100
True       89
Name: assists, dtype: int64

In [90]:
players_stats[players_stats['assists'].isnull()]

,date,comp,round,opponent,minutes,goals,assists,shots_total,shots_on_target,crosses,fouled,pens_made,pens_att,xg,npxg,xa,name
2197,2019-07-20,SuperLiga,Regular Season,Javor Ivanjica,90.0,1,NaN,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,El Fardou Ben
2199,2019-07-27,SuperLiga,Regular Season,Radnički Niš,90.0,1,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,El Fardou Ben
2203,2019-09-25,SuperLiga,Regular Season,Voždovac,90.0,1,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,El Fardou Ben
2204,2019-09-28,SuperLiga,Regular Season,Mačva Šabac,90.0,1,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,El Fardou Ben
2205,2019-10-06,SuperLiga,Regular Season,Napredak Kruš,82.0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,El Fardou Ben
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4017,2019-12-08,SuperLiga,Regular Season,Rad,90.0,1,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,Seydouba Soumah
4019,2019-12-15,SuperLiga,Regular Season,Radnik Surdulica,72.0,1,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,Seydouba Soumah
4020,2020-02-16,SuperLiga,Regular Season,Radnik Surdulica,90.0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,Seydouba Soumah
4021,2020-02-22,SuperLiga,Regular Season,TSC Bačka Top,90.0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,Seydouba Soumah


In [91]:
players_stats['minutes'].isnull().value_counts()

False    5179
True       10
Name: minutes, dtype: int64

In [92]:
players_stats[players_stats['minutes'].isnull()]

,date,comp,round,opponent,minutes,goals,assists,shots_total,shots_on_target,crosses,fouled,pens_made,pens_att,xg,npxg,xa,name
703,2019-09-22,Super League,Matchweek 4,Panathinaikos,NaN,0,0.0,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,Youssef El Arabi
713,2019-11-23,Super League,Matchweek 11,Panionios,NaN,0,0.0,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,Youssef El Arabi
977,2019-12-21,Premier League,Matchweek 18,Leicester City,NaN,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,Sergio Agüero
1033,2019-11-30,NaN,Matchweek 13,Ankaragücü,NaN,0,0.0,0.0,0.0,0.0,0.0,0,0,NaN,NaN,NaN,Papiss Demba Cissé
2126,2019-12-15,Premiership,Regular Season,Motherwell,NaN,0,0.0,0.0,0.0,0.0,0.0,0,0,NaN,NaN,NaN,Jermain Defoe
2649,2019-11-14,WCQ — AFC (M),Second round,Iraq,NaN,0,0.0,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,Kaveh Rezaei
3926,2020-01-29,EFL Cup,Semi-finals,Manchester Utd,NaN,0,0.0,0.0,0.0,0.0,1.0,0,0,NaN,NaN,NaN,Gabriel Jesus
4189,2019-11-06,Champions Lg,Group stage,Shakhtar,NaN,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,Damian Kadzior
4261,2019-12-21,Ligue 1,Matchweek 19,Lille,NaN,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,Islam Slimani
4894,2019-08-22,Europa Lg,Play-off Round,Qarabağ Ağdam,NaN,0,0.0,0.0,0.0,0.0,0.0,0,0,NaN,NaN,NaN,Kirk Millar


**We have made the choice to convert:**
* Games stats → integer 
* date → datetime

For the moment we have left the excepted missing stats (xg, npxg,xa) 

In [93]:
int_column = players_stats.columns[4:13]
players_stats[int_column] = players_stats[int_column].fillna(0)
players_stats[int_column] = players_stats[int_column].astype('int64')

In [94]:
players_stats['date'] = pd.to_datetime(players_stats['date'])

In [95]:
players_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5189 entries, 0 to 5188
Data columns (total 17 columns):
date               5189 non-null datetime64[ns]
comp               4325 non-null object
round              5189 non-null object
opponent           5189 non-null object
minutes            5189 non-null int64
goals              5189 non-null int64
assists            5189 non-null int64
shots_total        5189 non-null int64
shots_on_target    5189 non-null int64
crosses            5189 non-null int64
fouled             5189 non-null int64
pens_made          5189 non-null int64
pens_att           5189 non-null int64
xg                 1759 non-null float64
npxg               1759 non-null float64
xa                 1759 non-null float64
name               5189 non-null object
dtypes: datetime64[ns](1), float64(3), int64(9), object(4)
memory usage: 689.3+ KB


### Handle the missing values in Competition columns

In [96]:
players_stats[['round','comp']].isnull().sum()

round      0
comp     864
dtype: int64

In [97]:
players_stats[players_stats['comp'].isnull()][-50:]

,date,comp,round,opponent,minutes,goals,assists,shots_total,shots_on_target,crosses,fouled,pens_made,pens_att,xg,npxg,xa,name
4688,2019-08-24,NaN,Regular Season,Sint-Truiden,90,0,0,0,0,0,0,0,0,NaN,NaN,NaN,Cyle Larin
4689,2019-09-01,NaN,Regular Season,Antwerp,90,0,0,0,0,0,0,0,0,NaN,NaN,NaN,Cyle Larin
4690,2019-09-14,NaN,Regular Season,Eupen,90,1,0,0,0,0,0,0,0,NaN,NaN,NaN,Cyle Larin
4691,2019-09-22,NaN,Regular Season,Gent,90,1,0,0,0,0,0,0,0,NaN,NaN,NaN,Cyle Larin
4692,2019-09-28,NaN,Regular Season,Excel Mouscron,90,0,0,0,0,0,0,0,0,NaN,NaN,NaN,Cyle Larin
4693,2019-10-05,NaN,Regular Season,Cercle Brugge,90,2,2,0,0,0,0,0,0,NaN,NaN,NaN,Cyle Larin
4694,2019-10-19,NaN,Regular Season,Kortrijk,90,0,0,0,0,0,0,0,0,NaN,NaN,NaN,Cyle Larin
4695,2019-10-26,NaN,Regular Season,Oostende,90,0,0,0,0,0,0,0,0,NaN,NaN,NaN,Cyle Larin
4696,2019-10-30,NaN,Regular Season,Club Brugge,90,0,0,0,0,0,0,0,0,NaN,NaN,NaN,Cyle Larin
4697,2019-11-02,NaN,Regular Season,Waasland-Bev,90,1,0,0,0,0,0,0,0,NaN,NaN,NaN,Cyle Larin


🤔 The nan value in the Competion columns seems to be the pre saison games and also few asian matchweek game. I decided to drop this rows

players_stats.shape

In [98]:
players_stats.dropna(subset=['comp'], inplace=True)

In [99]:
players_stats.shape

(4325, 17)

## b. No Duplicated Values

In [100]:
players_stats.duplicated(['date','name']).value_counts()

False    4325
dtype: int64

---
# 3. Data exploration
<a id='explore'></a>

In [101]:
sum_stats = players_stats.groupby('name', sort=False).sum()
sum_stats.sort_values(by=['goals'],ascending=False)

,minutes,goals,assists,shots_total,shots_on_target,crosses,fouled,pens_made,pens_att,xg,npxg,xa
name,,,,,,,,,,,,
Robert Lewandowski,3351,43,7,173,84,22,54,4,4,28.9,25.9,6.8
Cristiano Ronaldo,3359,36,4,224,84,41,59,11,11,19.4,14.1,3.4
Erling Haaland,2176,36,6,96,51,15,26,3,3,11.2,9.6,0.9
Ciro Immobile,2941,33,8,126,49,12,40,11,13,21.8,13.2,5.7
Kylian Mbappé,2569,30,13,134,75,64,39,0,0,21.9,21.9,11.2
...,...,...,...,...,...,...,...,...,...,...,...,...
Yonatan Cohen,293,1,2,0,0,1,0,0,0,0.0,0.0,0.0
Besart Ibraimi,245,0,0,0,0,0,0,0,0,0.0,0.0,0.0
Mersudin Ahmetovic,217,0,0,0,0,0,0,0,0,0.0,0.0,0.0


🤔 weird some players have only one goals and 2 assists. Try to compare it with the top scorrer values:

In [102]:
join = top_scorer.join(sum_stats, on='name', how='inner', rsuffix='_total')
join.reset_index(drop=True, inplace=True)
join[['name','goals','goals_total']][:10]

,name,goals,goals_total
0,Ciro Immobile,27,33
1,Erling Haaland,25,36
2,Robert Lewandowski,25,43
3,Shon Weissman,22,24
4,Odsonne Edouard,21,27
5,Timo Werner,21,28
6,Cristiano Ronaldo,21,36
7,Ivan Trickovski,20,5
8,Alexander Sörloth,19,25
9,Lionel Messi,19,26


💡 Hypothesis: maybe one of the the first bot scraper only count domestic league goals

## Attribute Combinations
    - create bucket for the different game type (Domestic league,Champions league,Europa league,Cup)
    - numbered the game on each month

➤ Create bucket for the game type

In [103]:
players_stats['game_type'] =  ''
players_stats.loc[players_stats['round'].str.contains('Matchweek'),'game_type'] = 'Domestic league'
players_stats.loc[players_stats['round'].str.contains('Regular Season'),'game_type'] = 'Domestic league'
players_stats.loc[players_stats['comp']=='Champions Lg','game_type'] = 'Champions league'
players_stats.loc[players_stats['comp']=='Europa Lg','game_type'] = 'Europa league'
players_stats['game_type'].replace('','Cup',inplace=True)

In [104]:
stat_without_cup = players_stats[players_stats['game_type'] == 'Domestic league']
regular_stat = stat_without_cup.groupby('name').sum()
regular_stat.sort_values(by=['goals'],ascending=False)

,minutes,goals,assists,shots_total,shots_on_target,crosses,fouled,pens_made,pens_att,xg,npxg,xa
name,,,,,,,,,,,,
Ciro Immobile,2186,27,7,90,39,9,29,10,11,18.8,10.2,5.3
Robert Lewandowski,2041,25,3,106,48,15,34,3,3,20.8,18.6,5.2
Erling Haaland,1497,25,5,72,38,10,22,1,1,4.9,4.9,0.4
Shon Weissman,1814,22,2,78,43,11,23,1,1,0.0,0.0,0.0
Odsonne Edouard,2110,21,9,100,45,2,25,0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
Dan Nistor,1720,4,10,0,0,0,0,1,1,0.0,0.0,0.0
Christopher Nkunku,1451,4,12,39,17,96,29,0,0,5.3,5.3,7.8
Luis Alberto,2153,4,12,72,21,114,25,1,1,5.2,4.5,7.7


In [105]:
# check if the data now corresping between the two bot scraper
join = top_scorer.join(regular_stat, on='name', how='inner', rsuffix='_bot2')
join.reset_index(drop=True, inplace=True)
join[['name','goals','goals_bot2']][:10]

,name,goals,goals_bot2
0,Ciro Immobile,27,27
1,Erling Haaland,25,25
2,Robert Lewandowski,25,25
3,Shon Weissman,22,22
4,Odsonne Edouard,21,21
5,Timo Werner,21,21
6,Cristiano Ronaldo,21,21
7,Alexander Sörloth,19,19
8,Lionel Messi,19,19
9,Jamie Vardy,19,19


Goals now corresponds it's work 🥳

In [106]:
# add the league_country
players_stats = pd.merge(players_stats, top_scorer[['name','league']], how='left')

➤ add the players' club

In [107]:
players_stats = pd.merge(players_stats, top_scorer[['name','club']], how='left')

➤ Numbered the game per month for each player

In [108]:
month = players_stats['date'].dt.month
players_stats['game_number_per_month'] = players_stats.groupby(['name',month]).cumcount()+1

In [109]:
# test check
players_stats[:10][['date','game_number_per_month','name']]

,date,game_number_per_month,name
0,2019-08-25,1,Ciro Immobile
1,2019-09-01,1,Ciro Immobile
2,2019-09-08,2,Ciro Immobile
3,2019-09-15,3,Ciro Immobile
4,2019-09-22,4,Ciro Immobile
5,2019-09-25,5,Ciro Immobile
6,2019-09-29,6,Ciro Immobile
7,2019-10-03,1,Ciro Immobile
8,2019-10-06,2,Ciro Immobile
9,2019-10-12,3,Ciro Immobile


---
# 4. Save
<a id='save'></a>

In [110]:
# clean a little bit
players_stats.drop(columns=['round','min_per_goal_x','min_per_assit_x'],inplace=True, errors='ignore')
players_stats[:5]

,date,comp,opponent,minutes,goals,assists,shots_total,shots_on_target,crosses,fouled,pens_made,pens_att,xg,npxg,xa,name,game_type,league,club,game_number_per_month
0,2019-08-25,Serie A,Sampdoria,90,2,0,5,4,0,3,0,0,1.3,1.3,0.1,Ciro Immobile,Domestic league,Italy,Lazio,1
1,2019-09-01,Serie A,Roma,90,0,1,7,1,0,1,0,0,0.6,0.6,0.7,Ciro Immobile,Domestic league,Italy,Lazio,1
2,2019-09-08,Euro Qualifying,Finland,76,1,0,5,2,0,0,0,0,0.5,0.5,0.1,Ciro Immobile,Cup,Italy,Lazio,2
3,2019-09-15,Serie A,SPAL,90,1,0,0,0,1,0,1,1,0.8,0.0,0.1,Ciro Immobile,Domestic league,Italy,Lazio,3
4,2019-09-22,Serie A,Parma,63,1,0,3,2,2,0,0,0,0.2,0.2,0.2,Ciro Immobile,Domestic league,Italy,Lazio,4


In [111]:
# players_stats.to_csv('../database.csv')

**➤ save to a google spreadsheet**

In [112]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

In [113]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'drive_key.json', scope)
gc = gspread.authorize(credentials)

In [114]:
spreadsheet_key = '10EDLlCx53daOsInMuCSSxqTvVz4O1z7ijj6WcO3PW8Q'
wks_name = 'Master'
d2g.upload(players_stats, spreadsheet_key, wks_name, credentials=credentials, row_names=True)

<Worksheet 'Master' id:0>

In [116]:
top_scorer

,name,club,age,league,nationality,match_played,goals,assists,total
0,Ciro Immobile,Lazio,30,Italy,Italy,26,27,7,34
13,Erling Haaland,Bor. Dortmund,19,Germany,Norway,22,25,8,33
3,Robert Lewandowski,Bayern Munich,31,Germany,Poland,23,25,3,28
6,Shon Weissman,Wolfsberger AC,24,Austria,Israel,21,22,3,25
1,Odsonne Edouard,Celtic,22,Scotland,France,27,21,12,33
...,...,...,...,...,...,...,...,...,...
147,Luis Alberto,Lazio,27,Italy,Spain,25,4,13,17
231,Miroslav Stevanovic,Servette FC,29,Switzerland,Bosnia-Herzegovina,21,4,11,15
243,Dan Nistor,CS U Craiova,31,Romania,Romania,22,4,11,15
119,Christopher Nkunku,RB Leipzig,22,Germany,France,23,4,14,18


In [119]:
# test 
players_stats['league'].unique()

array(['Italy', 'Germany', 'Austria', 'Scotland', 'Cyprus', 'Turkey',
       'Spain', 'England', 'Belgium', 'France', 'Switzerland', 'Monaco',
       'Ukraine', 'Greece', 'Croatia', 'Mexico', 'Poland', 'Netherlands',
       'Australia', 'Luxembourg', 'United States', 'Portugal',
       'Gibraltar', 'Saudi Arabia', 'South Africa', 'Bosnia-Herzegovina',
       'Serbia', 'Russia', 'Wales', 'Slovenia', 'Czech Republic',
       'Romania', 'Israel', 'Denmark', 'Malta', 'Northern Ireland',
       'North Macedonia'], dtype=object)

,date,comp,opponent,minutes,goals,assists,shots_total,shots_on_target,crosses,fouled,pens_made,pens_att,xg,npxg,xa,game_type,league,club,game_number_per_month
name,,,,,,,,,,,,,,,,,,,
Adis Jahovic,28,28,28,28,28,28,28,28,28,28,28,28,0,0,0,28,28,28,28
Adrian Cieslewicz,5,5,5,5,5,5,5,5,5,5,5,5,0,0,0,5,5,5,5
Alan Pulido,20,20,20,20,20,20,20,20,20,20,20,20,0,0,0,20,20,20,20
Alassane Plea,28,28,28,28,28,28,28,28,28,28,28,28,27,27,27,28,28,28,28
Aleksandr Sobolev,21,21,21,21,21,21,21,21,21,21,21,21,0,0,0,21,21,21,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wissam Ben Yedder,35,35,35,35,35,35,35,35,35,35,35,35,26,26,26,35,35,35,35
Yonatan Cohen,5,5,5,5,5,5,5,5,5,5,5,5,0,0,0,5,5,5,5
Youssef El Arabi,38,38,38,38,38,38,38,38,38,38,38,38,9,9,9,38,38,38,38
